In [7]:
wdir = "./"
import shutil

In [8]:
import pandas as pd
def getPt(inpath, station, network, channel):
    df = pd.read_csv(f"{inpath}/arrivals.txt", header = None, delim_whitespace=True, skiprows = 1, names=[ "network", "station", 
                                                                                        "channel", "two-digit location code", 
                                                                                        "latitude", "longitude", "elevation", 
                                                                                        "phase", "first- motion", "signal onset quality", 
                                                                                        "pick quality", "epicentral distance",  
                                                                                        "time after origin time"])
    nsc = df.loc[(df['station'] == station) & (df['network'] == network) & (df['phase'] == "P")]
    #print(df)
    #print(station, network, channel)
    with open(f'{inpath}/arrivals.txt') as f:
        first_line = f.readline()
    evt_id = first_line.split()[0]
    if len(nsc["time after origin time"]) == 0:
        #print("no hand picked wave time")
        return [evt_id, False]
    else:
        t = nsc["time after origin time"].iloc[0]
        #print("t", t)
        return [evt_id, t]

In [18]:
#  refine P-wave arrival and calculate theoretical sP-P differential time
import os
import glob
import obspy
import time
from obspy.taup import TauPyModel
from obspy.clients.iris import Client
client = Client()
import matplotlib.pyplot as plt
import numpy as np
from obspy.signal.trigger import classic_sta_lta
from IPython.display import clear_output
switch1 = True
switch2 = True

data_sac="./Download/SAC"

file2 = open("./progression-autofilter.info", "r")

overall_progress = file2.read().split()
evt = overall_progress[1]
sacfile_name = overall_progress[0]


file2.close()
total = len(os.listdir(data_sac))
events = os.listdir(data_sac)
for ev in events:
        #print(ev)
        progress = [events.index(ev), len(events), ev]
        print("progress:", progress)
        if ev != evt and switch1 == True:
            continue
        switch1 = False

        inpath = os.path.join(data_sac, ev)
        if len(glob.glob(f"{inpath}/*R.SAC.bp.cut")) == 0:
            continue
        else:            
            funiq = []
            for sac in glob.glob(f"{inpath}/*R.SAC.bp.cut"):
                st = obspy.read(sac)
                funiq.append("{}.{}.{}".format(st[0].stats.network,st[0].stats.station,st[0].stats.location))
            fU = sorted(set(funiq))
            subTotal = len(fU)
           
            for sacfile in fU:
                
                iteration2 = iter(fU)
                
                if switch2 == True and sacfile != sacfile_name:
                    continue
                switch2 = False
                
#                 if fU.index(sacfile) % 30 == 0:    
#                     progress = [fU.index(sacfile), len(fU), events.index(ev), len(events), sacfile, ev]
#                     print("progress:", progress)
                file2 = open("./progression-autofilter.info", "w")
                file2.write(f" {sacfile} {ev}")
                file2.close()
                
                st = obspy.read(f"{inpath}/{sacfile}*[RTZ].SAC.bp.cut")
                
                #get hand-picked time
                network = st[0].stats.network
                station = st[0].stats.station
                channel = st[0].stats.channel
                
                handpick_info = getPt(inpath, station, network, channel)
                handpick_t = handpick_info[1]
                evt_id = handpick_info[0]
                
#                 shift = st[0].stats.sac.o
                #print(shift)

                ### refine P-wave arrival using sta/lta method
#                 df = st[0].stats.sac.delta
#                 t = np.arange(st[0].stats.sac.t1-5, st[0].stats.sac.t1+10, st[0].stats.delta)
                
                if handpick_t == False:
                    continue
                # time window
#                 idx1 = int((st[0].stats.sac.t1-5-st[0].stats.sac.b)/st[0].stats.delta)
#                 idx2 = int((st[0].stats.sac.t1+10-st[0].stats.sac.b)/st[0].stats.delta)
                Tp = handpick_t

                ### calculate the theoretical sP-P differential time
                model = TauPyModel(model="./hk.npz")
                sachd = st[0].stats.sac
                distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
                gcarc = distaz['distance']
                
                #print("Event depth: ", sachd["evdp"])
                #print("Distance (degrees): ", gcarc)
                
                if sachd["evdp"] < 0:
                    arrival = model.get_travel_times(source_depth_in_km=0,distance_in_degree=gcarc, phase_list=["sPvmP"])
                else:                                  
                    arrival = model.get_travel_times(source_depth_in_km=sachd["evdp"],distance_in_degree=gcarc, phase_list=["sPvmP"])
                #print(arrival)
                try:
                    TsPmP = float("{:.3f}".format(arrival[0].time))
                except: 
                    continue
                arrival = model.get_travel_times(source_depth_in_km=sachd["evdp"],distance_in_degree=gcarc, phase_list=["sPn"])
                if len(arrival) != 0:
                    TsPn = float("{:.3f}".format(arrival[0].time))
                else:
                    TsPn = "nan"
                
                file = open("./judge-handpickP.info", "a")
                file.write("{} {} {} {} {}\n".format(ev, sacfile, Tp, TsPmP, TsPn))
                file.close()
                 
            src_path = f"./judge-handpickP.info"
            if not os.path.exists(f"{data_sac}/{ev}/judge"):
                os.mkdir(f"{data_sac}/{ev}/judge")            
            dst_path = f"{data_sac}/{ev}/judge/handpickP.info"
            try:
                shutil.move(src_path, dst_path)
            except:
                print("no sPmP arrival for event: {}".format(ev))
                continue
            

progress: [0, 514, '20220102001614080']
progress: [1, 514, '20220102232745920']
progress: [2, 514, '20220103023448200']
progress: [3, 514, '20220104090504520']
progress: [4, 514, '20220104124219810']
progress: [5, 514, '20220105012439450']
progress: [6, 514, '20220106035747810']
progress: [7, 514, '20220106184335940']
progress: [8, 514, '20220107044318400']
progress: [9, 514, '20220107045132740']
progress: [10, 514, '20220107124239920']
progress: [11, 514, '20220107131511710']
progress: [12, 514, '20220107150647440']
progress: [13, 514, '20220108062131650']
progress: [14, 514, '20220108062454860']
progress: [15, 514, '20220108223706690']
progress: [16, 514, '20220108234618060']
progress: [17, 514, '20220108235154580']
progress: [18, 514, '20220109035652870']
progress: [19, 514, '20220109200252070']
progress: [20, 514, '20220110185947040']
progress: [21, 514, '20220111054724830']
progress: [22, 514, '20220111114149810']
progress: [23, 514, '20220112002458770']
progress: [24, 514, '20220

no sPmP arrival for event: 20220313205132570
progress: [197, 514, '20220313213335780']
progress: [198, 514, '20220314054039570']
progress: [199, 514, '20220314055945990']
progress: [200, 514, '20220314191616300']
progress: [201, 514, '20220315005731560']
progress: [202, 514, '20220315005902100']
progress: [203, 514, '20220315132335010']
no sPmP arrival for event: 20220315132335010
progress: [204, 514, '20220315181534880']
progress: [205, 514, '20220316031444040']
no sPmP arrival for event: 20220316031444040
progress: [206, 514, '20220316054313670']
progress: [207, 514, '20220316060858910']
progress: [208, 514, '20220316101338140']
progress: [209, 514, '20220316170510520']
progress: [210, 514, '20220316175102390']
progress: [211, 514, '20220316234635830']
progress: [212, 514, '20220317014656550']
progress: [213, 514, '20220317045350680']
progress: [214, 514, '20220317184856910']
progress: [215, 514, '20220317202110650']
progress: [216, 514, '20220318092715930']
progress: [217, 514, '202

progress: [386, 514, '20220511143632960']
progress: [387, 514, '20220511171942950']
progress: [388, 514, '20220511222602830']
progress: [389, 514, '20220513051138470']
progress: [390, 514, '20220513133236610']
progress: [391, 514, '20220513162842500']
progress: [392, 514, '20220513194518040']
progress: [393, 514, '20220514001111010']
progress: [394, 514, '20220514212814790']
progress: [395, 514, '20220515144645220']
progress: [396, 514, '20220516061650290']
progress: [397, 514, '20220516063652230']
progress: [398, 514, '20220516103107280']
progress: [399, 514, '20220516153638810']
progress: [400, 514, '20220517012122980']
progress: [401, 514, '20220517041514090']
progress: [402, 514, '20220517064728530']
progress: [403, 514, '20220518154752730']
progress: [404, 514, '20220519011841930']
progress: [405, 514, '20220519053359940']
progress: [406, 514, '20220519125200230']
progress: [407, 514, '20220520002443720']
progress: [408, 514, '20220520115355270']
progress: [409, 514, '202205201810

In [10]:
print(src_path, dst_path)

.//judge-handpickP.info ./Download/SAC/20220202161103660/judge/handpickP.info
